In [ ]:
import pandas as pd
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
import pandas as pd
import xgboost
scaler = StandardScaler()
df = pd.read_csv('data/Train.csv')
df_test = pd.read_csv('data/Submission.csv')
df['Electricity_Billing_Date'] = pd.to_datetime(df['Electricity_Billing_Date'])

# 날짜에서 유용한 특징 추출 (예: 년도와 월)
df['Billing_Year'] = df['Electricity_Billing_Date'].dt.year
df['Billing_Month'] = df['Electricity_Billing_Date'].dt.month
df = pd.get_dummies(df, columns = ['House_Type', 'Billing_Year', 'Billing_Month'])

# 더 이상 사용하지 않는 Billing_Date 열 삭제
df.drop('Electricity_Billing_Date', axis=1, inplace=True)


print(f'train dataset number of columns: {len(df.columns)}')
			
y = df['Electricity_Bill']
X = df.drop('Electricity_Bill', axis =1)

X = scaler.fit_transform(X)

linreg = xgboost.XGBRegressor()

scores = cross_val_score(linreg, X, y)
print(f'cv scores:{scores}')

linreg2 = xgboost.XGBRegressor()
linreg2.fit(X, y)


df_test['Electricity_Billing_Date'] = pd.to_datetime(df_test['Electricity_Billing_Date'])

# 날짜에서 유용한 특징 추출 (예: 년도와 월)
df_test['Billing_Year'] = df_test['Electricity_Billing_Date'].dt.year
df_test['Billing_Month'] = df_test['Electricity_Billing_Date'].dt.month
df_test = pd.get_dummies(df_test, columns = ['House_Type', 'Billing_Year', 'Billing_Month'])

# 더 이상 사용하지 않는 Billing_Date 열 삭제
df_test.drop('Electricity_Billing_Date', axis=1, inplace=True)
test_X = df_test.drop('Electricity_Bill', axis =1)
print(f'test dataset number of columns: {len(test_X.columns)}')
test_X = scaler.fit_transform(test_X)
test_y = linreg2.predict(test_X)

df_predict = pd.DataFrame()
df_predict['Predicted_Electricity_Bill'] = test_y
print(df_predict)
df_predict.to_csv('data/answer.csv')

'''
from sklearn.model_selection import StratifiedKFold, cross_val_score

# StratifiedKFold 적용 (분류 문제에서는 일반적이지만 회귀 문제에도 사용할 수 있음)
cv = StratifiedKFold(n_splits=5)
scores = cross_val_score(model, X, y, cv=cv)

from sklearn.model_selection import KFold

# KFold 교차 검증에서 폴드 수 증가
kf = KFold(n_splits=10, shuffle=True, random_state=42)
scores = cross_val_score(model, X, y, cv=kf)

'''
